# Init

In [ ]:
import os, sys

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

config = "bruz"
if config == "ur1":
    sys.path.insert(0, "../")
    siepelDir = "C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2"
elif config == "bruz":
    sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
    siepelDir = "/home/pleroy/DATA/CAMILL/WR15_v2"

import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.Probe import Probe
from toolsdiadem.ConfigSer import ConfigSer

In [ ]:
#%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [ ]:
cmap='jet'
configSupport = os.path.join(siepelDir, "wr10_plaqueSupport.ini")
configRef = os.path.join(siepelDir, "wr10_plaqueRef.ini")
confSupport = ConfigSer(configSupport)
confRef = ConfigSer(configRef)
confSupport.printConfig()
confRef.printConfig()

# Données de la sonde

In [ ]:
probe = Probe(os.path.join(confRef.base_path, "20200707/S11FDC_WR10.s1p"))
probe_interpolation = interpolate.interp1d(probe.freq, probe.S11)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(probe.d, ser.dB(np.fft.ifft(probe.S11)), label=f"Probe" )
ax.legend()
ax.grid()

# Données des plaques

## Référence

In [ ]:
dir_ref = f"{confRef.base_path}/20200707/031_plaqueRef"
freq_reference, S11_reference = ser.getData_ssb(
    confRef.nb_elev, confRef.nb_freq, confRef.nb_ssb, dir_ref, tag="WR10_")

In [ ]:
probe_S11_interp = probe_interpolation(freq_reference)
d = np.linspace(0, 300 / (freq_reference[1] - freq_reference[0]), confRef.nb_freq) * 0.5;

In [ ]:
S11_reference_compensated = S11_reference - probe_S11_interp

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolormesh(confRef.elevation, freq_reference, ser.dB(S11_reference).T, cmap=cmap )
title = f"S11_reference"
ax0.set_title(title)
ax0.set_xlabel( "Elévation [°]" )
ax0.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax0, aspect='auto' )

im = ax1.pcolormesh(confRef.elevation, freq_reference, ser.dB(S11_reference_compensated).T, cmap=cmap )
title = f"S11_reference_compensated"
ax1.set_title(title)
ax1.set_xlabel( "Elévation [°]" )
ax1.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax1, aspect='auto' )

## Support

In [ ]:
dir_support = f"{confSupport.base_path}/20200709/035_plaqueSupport"
freq_support, S11_support = ser.getData_ssb(
    confSupport.nb_elev, confSupport.nb_freq, confSupport.nb_ssb, dir_support, tag="WR10_")

In [ ]:
S11_support_compensated = S11_support - probe_S11_interp

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolormesh(confSupport.elevation, freq_support, ser.dB(S11_support).T, cmap=cmap )
title = f"S11_support"
ax0.set_title(title)
ax0.set_xlabel( "Elévation [°]" )
ax0.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax0, aspect='auto' )

im = ax1.pcolormesh(confSupport.elevation, freq_support, ser.dB(S11_support_compensated).T, cmap=cmap )
title = f"S11_support_compensated"
ax1.set_title(title)
ax1.set_xlabel( "Elévation [°]" )
ax1.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax1, aspect='auto' )

# Recherche de maximum

## Référence

In [ ]:
idx = np.where(abs(d - confSupport.gateCenter) < confSupport.gateWidth)[0]

In [ ]:
maxVals = np.amax(np.abs(np.fft.ifft(S11_reference, axis=1))[:,idx], axis=1)
maxElev = np.where(maxVals == np.amax(maxVals))
print(f"maxElev = {maxElev}")
iElevRefMax = maxElev[0][0]
print(f"reference elevation at maximum: {confRef.elevation[iElevRefMax]:.2f}, index {iElevRefMax}")

## Support

In [ ]:
maxVals = np.amax(np.abs(np.fft.ifft(S11_support, axis=1))[:,idx], axis=1)
maxElev = np.where(maxVals == np.amax(maxVals))
print(f"maxElev = {maxElev}")
iElevSupportMax = maxElev[0][0]
print(f"support elevation at maximum: {confSupport.elevation[iElevSupportMax]:.2f}, index {iElevSupportMax}")

## Plot

In [ ]:
reference_max = S11_reference[iElevRefMax,:]
support_max = S11_support[iElevSupportMax,:]
referenceComp_max = S11_reference_compensated[iElevRefMax,:]
supportComp_max = S11_support_compensated[iElevSupportMax,:]

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(d, ser.dB(np.fft.ifft(reference_max)), label=f"Plaque_max, {confRef.elevation[iElevRefMax]}")
ax.plot(d, ser.dB(np.fft.ifft(referenceComp_max)), label=f"PlaqueComp_max, {confRef.elevation[iElevRefMax]}")
ax.plot(d, ser.dB(np.fft.ifft(support_max)), label=f"Support_max, {confSupport.elevation[iElevSupportMax]:.2f}")
ax.plot(d, ser.dB(np.fft.ifft(supportComp_max)), label=f"SupportComp_max, {confSupport.elevation[iElevSupportMax]:.2f}")
ax.legend()
ax.grid()

# Save data

In [ ]:
tmp = np.zeros((confRef.nb_freq, 3))
tmp[:,0] = freq_reference
tmp[:,1] = np.real(reference_max)
tmp[:,2] = np.imag(reference_max)
filename = f"{confRef.out_path}/data/S11_plaque_ref_from_scan.txt"
print(filename)
np.savetxt(filename, tmp, header="GHz real imag")

In [ ]:
tmp = np.zeros((confRef.nb_freq, 3))
tmp[:,0] = freq_reference
tmp[:,1] = np.real(probe_S11_interp)
tmp[:,2] = np.imag(probe_S11_interp)
filename = f"{confRef.out_path}/data/S11_probe_interpolated.txt"
print(filename)
np.savetxt(filename, tmp, header="GHz real imag")